In [7]:
import urllib2
import time
from bs4 import BeautifulSoup
import pandas as pd

def get_content1(url):

    html = urllib2.urlopen(url)

    return BeautifulSoup(html,"lxml")

In [49]:
from selenium import webdriver

def get_content(url):
    browser = webdriver.Chrome("/Users/Admin/Documents/WebDrivers/chromedriver")
    browser.get(url)
    
    # scroll down and wait
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    
    html = browser.page_source

    return BeautifulSoup(html, "lxml"), browser

In [48]:
def get_counties(soup):
    counties = []
    for s in soup.findAll("div", { "class" : "title" }):
        county = s.string
        if county != None:
            if not 'Reporting' in county:
                counties.append(str(s.string))
    return counties

In [64]:
def data_candidate(df, soup, counties, state):
    
    candidate_vote = []
    for i, t in enumerate(soup.findAll("table", { "class" : "results-table" })[1:]):
       
        cand = []
        for s in t.findAll("span", { "class" : "name-combo" }):
            cand.append(s.text.split()[-1])

        number = []
        for n in t.findAll("span", { "class" : "number" }):
            number.append(n.text.replace("%", ""))
        
        votes = []
        for v in t.findAll("td", { "class" : "results-popular" }):

            votes.append(float(v.text.replace(',','')))



        for j in range(len(cand)):
            candidate_vote.append([state, counties[i], cand[j], number[j], votes[j]]) 
            


    temp_df = pd.DataFrame(candidate_vote, columns=['State', 'County', 'Candidate', 'Votes', 'NVotes'] )
    print len(temp_df['County'].unique())

    return df.append(temp_df)

In [40]:
df = pd.DataFrame(columns=['State', 'County', 'Candidate', 'Votes', 'NVotes'])

In [41]:
s2abr = pd.read_csv('data/state_to_abbr.csv')
s2abr['state'] = s2abr['state'].apply(lower)
s2abr = s2abr.set_index('state')
s2abr = s2abr.to_dict()['abbr']

In [42]:
def lower(x):
    return x.lower()
def change(x):
    if x in s2abr.keys():
        return s2abr[x] 
    else:
        return 'DC'

In [43]:
df_complete = pd.read_csv('data/raw/complete_county.csv')
print len(df_complete)
# change state name to abbreviation
df_complete['State'] = df_complete['State'].map(lambda x: x.replace('_', ''))
df_complete['State'] =  df_complete['State'].map(lower)
df_complete['State'] = df_complete['State'].map(change)

3143


In [65]:
for state in s2abr.keys():
   
    print state
    print 'Expected counties:', len(df_complete[df_complete['State'] == s2abr[state]]['County'].unique())
    
    state = state.lower()
    state = state.replace(" ", "-")
    
    soup, browser = get_content("http://www.politico.com/2016-election/results/map/president/{}".format(state))
    counties = get_counties(soup)
    df = data_candidate(df, soup, counties, state)
    df.to_csv("data/raw/vote_by_county.csv")
    browser.quit()

wyoming
Expected counties: 23
23
colorado
Expected counties: 64
64
washington
Expected counties: 39
39
hawaii
Expected counties: 5
4
tennessee
Expected counties: 95
95
wisconsin
Expected counties: 72
72
nevada
Expected counties: 17
17
maine
Expected counties: 16
16
north dakota
Expected counties: 53
53
mississippi
Expected counties: 82
82
south dakota
Expected counties: 66
66
new jersey
Expected counties: 21
21
oklahoma
Expected counties: 77
77
delaware
Expected counties: 3
3
minnesota
Expected counties: 87
87
north carolina
Expected counties: 100
100
illinois
Expected counties: 102
102
new york
Expected counties: 62
62
arkansas
Expected counties: 75
75
indiana
Expected counties: 92
92
maryland
Expected counties: 24
24
louisiana
Expected counties: 64
64
idaho
Expected counties: 44
44
arizona
Expected counties: 15
15
iowa
Expected counties: 99
99
west virginia
Expected counties: 55
55
michigan
Expected counties: 83
83
kansas
Expected counties: 105
105
utah
Expected counties: 29
29
virgi

In [ ]:
redo_list = ['virginia', ]